In [1]:
import pathlib

import pandas as pd
from pycytominer import feature_select
from pycytominer.cyto_utils import output

In [2]:
# set path to normalized data
normalized_data_path = pathlib.Path(
    "../data/CP_scDINO_features/combined_CP_scDINO_norm.parquet"
).resolve(strict=True)

# set the outout file path
feature_selected_output_file_path = pathlib.Path(
    "../data/CP_scDINO_features/combined_CP_scDINO_norm_fs.parquet"
).resolve()

# read in the normalized data
normalized_data = pd.read_parquet(normalized_data_path)
print(normalized_data.shape)
normalized_data.head()

(145489, 3869)


,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,Metadata_ImageNumber,Metadata_FOV,Metadata_Time,Metadata_Cells_Number_Object_Number,...,channel_DNA_cls_feature_91_scDINO,channel_DNA_cls_feature_92_scDINO,channel_DNA_cls_feature_93_scDINO,channel_DNA_cls_feature_94_scDINO,channel_DNA_cls_feature_95_scDINO,channel_DNA_cls_feature_96_scDINO,channel_DNA_cls_feature_97_scDINO,channel_DNA_cls_feature_98_scDINO,channel_DNA_cls_feature_99_scDINO,channel_DNA_cls_feature_9_scDINO
0,1.0,C-02,178.0,Staurosporine,0.0,negative,1,0001,0.0,98.0,...,0.187026,-0.248392,1.603370,0.312717,1.018808,-0.202042,0.147753,-0.357406,-0.140129,1.510029
1,1.0,C-02,178.0,Staurosporine,0.0,negative,1,0001,0.0,108.0,...,-1.215989,-0.610512,-1.242749,0.410329,0.872185,1.059767,-1.079649,0.219456,0.469064,-0.973322
2,1.0,C-02,178.0,Staurosporine,0.0,negative,1,0001,0.0,123.0,...,1.306198,1.369489,0.483010,-0.765297,0.035978,-0.131003,1.354683,-0.020082,2.435365,3.352972
3,1.0,C-02,178.0,Staurosporine,0.0,negative,1,0001,0.0,127.0,...,-0.524220,-0.256391,0.310584,-0.323113,-1.792923,0.823824,-0.372862,1.516754,0.890084,0.934903
4,1.0,C-02,178.0,Staurosporine,0.0,negative,1,0001,0.0,15.0,...,0.002674,1.370834,0.041910,0.942032,0.615786,-1.282878,1.124272,-0.043270,-1.727197,0.671195


In [3]:
# define operations to be performed on the data
# list of operations for feature select function to use on input profile
feature_select_ops = [
    "variance_threshold",
    "blocklist",
    "drop_na_columns",
    "correlation_threshold",
]

# Get columns that contain "Metadata"
metadata_features = normalized_data.columns[
    normalized_data.columns.str.contains("Metadata")
].tolist()

# get the feature columns
feature_columns = normalized_data.columns.difference(metadata_features).to_list()

In [4]:
manual_block_list = [
    "Nuclei_AreaShape_BoundingBoxArea",
    "Nuclei_AreaShape_BoundingBoxMinimum_X",
    "Cells_AreaShape_BoundingBoxArea",
]

In [ ]:
feature_select_df = feature_select(
    normalized_data,
    operation=feature_select_ops,
    features=feature_columns,
)
# add "Metadata_" to the beginning of each column name in the list
feature_select_df.columns = [
    "Metadata_" + column if column in manual_block_list else column
    for column in feature_select_df.columns
]
print("Feature selection complete, saving to parquet file!")
# save features selected df as parquet file
output(
    df=feature_select_df,
    output_filename=feature_selected_output_file_path,
    output_type="parquet",
)
print("Features have been selected!")
# check to see if the shape of the df has changed indicating feature selection occurred
print(normalized_data.shape)
print(feature_select_df.shape)
print(f"{normalized_data.shape[1] - feature_select_df.shape[1]} features were removed.")
print(f"{feature_select_df.shape[1]} features remain.")
feature_select_df.head()

Feature selection complete, saving to parquet file!
Features have been selected!
(145489, 3869)
(145489, 2449)
1420 features were removed.
2449 features remain.


,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,Metadata_ImageNumber,Metadata_FOV,Metadata_Time,Metadata_Cells_Number_Object_Number,...,channel_DNA_cls_feature_91_scDINO,channel_DNA_cls_feature_92_scDINO,channel_DNA_cls_feature_93_scDINO,channel_DNA_cls_feature_94_scDINO,channel_DNA_cls_feature_95_scDINO,channel_DNA_cls_feature_96_scDINO,channel_DNA_cls_feature_97_scDINO,channel_DNA_cls_feature_98_scDINO,channel_DNA_cls_feature_99_scDINO,channel_DNA_cls_feature_9_scDINO
0,1.0,C-02,178.0,Staurosporine,0.0,negative,1,0001,0.0,98.0,...,0.187026,-0.248392,1.603370,0.312717,1.018808,-0.202042,0.147753,-0.357406,-0.140129,1.510029
1,1.0,C-02,178.0,Staurosporine,0.0,negative,1,0001,0.0,108.0,...,-1.215989,-0.610512,-1.242749,0.410329,0.872185,1.059767,-1.079649,0.219456,0.469064,-0.973322
2,1.0,C-02,178.0,Staurosporine,0.0,negative,1,0001,0.0,123.0,...,1.306198,1.369489,0.483010,-0.765297,0.035978,-0.131003,1.354683,-0.020082,2.435365,3.352972
3,1.0,C-02,178.0,Staurosporine,0.0,negative,1,0001,0.0,127.0,...,-0.524220,-0.256391,0.310584,-0.323113,-1.792923,0.823824,-0.372862,1.516754,0.890084,0.934903
4,1.0,C-02,178.0,Staurosporine,0.0,negative,1,0001,0.0,15.0,...,0.002674,1.370834,0.041910,0.942032,0.615786,-1.282878,1.124272,-0.043270,-1.727197,0.671195


: 